In [24]:
import pytesseract
from PIL import Image
from tqdm import tqdm

import os
import csv

from concurrent.futures import ThreadPoolExecutor

In [25]:
files = os.listdir('../../data/photos')
header = ['Channel Username', 'Image ID', 'Content', 'Media Path']
print(len(files))

28789


In [26]:
def write_to_file(filename, rows, header=None):
    file_exists = os.path.exists('../../data/' + filename)
    write_header = not file_exists or os.path.getsize('../../data/' + filename) == 0

    with open('../../data/' + filename, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if write_header and header:
            writer.writerow(header)
        writer.writerows(rows)

In [27]:
def process_image(file):
    if file.endswith('.jpg') or file.endswith('.png'):
        img_path = os.path.join('../../data/photos', file)
        temp = file.split('_')
        channel_username = temp[0]
        image_id = temp[1].split('.')[0]
        media_path = file

        img = Image.open(img_path)
        text = pytesseract.image_to_string(img, lang='eng')
        return [channel_username, image_id, text, media_path]
    return None

In [28]:
rows = []
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_image, files), total=len(files)))
    rows = [row for row in results if row is not None]

write_to_file('extracted_text.csv', rows, header)

100%|██████████| 28789/28789 [32:14<00:00, 14.88it/s]  
